# Compute and plot identities from alignment

Import Python modules:

In [12]:
import itertools

import altair as alt

import Bio.SeqIO

import pandas as pd

import yaml

Get variables from `snakemake`:

In [4]:
if "snakemake" in globals() or "snakemake" in locals():
    # from snakemake
    inputfasta = snakemake.input.fasta
    chartfile = snakemake.output.chart
    csvfile = snakemake.output.csv
    alignment_ref = snakemake.params.alignment_ref
    ref_regions = snakemake.params.ref_regions
else:
    # manually define for running interactively
    inputfasta = "../results/viruses/all_viruses_aligned.fasta"
    chartfile = "../results/identities/identities.html"
    csvfile = "../results/identities/identities.csv"
    with open("../config.yaml") as f:
        config = yaml.safe_load(f)
    alignment_ref = config["alignment_ref"]
    ref_regions = config["ref_regions"]

Read alignment:

In [8]:
alignment = {s.id: str(s.seq) for s in Bio.SeqIO.parse(inputfasta, "fasta")}

Get reference to alignment numbering (1-based):

In [11]:
alignment_to_ref_numbering = {}
i_ref = 0
for i_alignment, nt in enumerate(alignment[alignment_ref], start=1):
    if nt != "-":
        i_ref += 1
    alignment_to_ref_numbering[i_alignment] = i_ref
    
ref_to_alignment_numbering = {y:x for x, y in alignment_to_ref_numbering.items()}

Compute identities for each gene:

In [23]:
records = []
for gene, (start, end) in ref_regions.items():
    for i, virus_1 in enumerate(alignment):
        seq_1 = alignment[virus_1][start - 1: end]
        for virus_2 in list(alignment)[i: ]:
            seq_2 = alignment[virus_2][start - 1: end]
            ident = n_w_gaps = n_no_gaps = 0
            for nt1, nt2 in zip(seq_1, seq_2):
                if nt1 == nt2 == "-":
                    pass
                elif (nt1 == "-") or (nt2 == "-"):
                    n_w_gaps += 1
                elif nt1 == nt2:
                    ident += 1
                    n_no_gaps += 1
                    n_w_gaps += 1
                else:
                    n_no_gaps += 1
                    n_w_gaps += 1
            records.append((gene, virus_1, virus_2, ident, n_no_gaps, n_w_gaps))
            if virus_1 != virus_2:
                records.append((gene, virus_1, virus_2, ident, n_no_gaps, n_w_gaps))
        
df = (
    pd.DataFrame(
        records,
        columns=[
            "gene",
            "virus_1",
            "virus_2",
            "n identities",
            "no gaps",
            "with gaps",
        ],
    )
    .melt(
        id_vars=["gene", "virus_1", "virus_2", "n identities"],
        value_vars=["no gaps", "with gaps"],
        var_name="count gaps as mismatches",
        value_name="n sites",
    )
    .query("`n sites` > 0")
    .assign(percent_identity=lambda x: 100 * x["n identities"] / x["n sites"])
)

print(f"Writing identites to {csvfile}")

df.to_csv(csvfile, index=False, float_format="%.5g")

df

,gene,virus_1,virus_2,n identities,count gaps as mismatches,n sites,percent_identity
0,ORF1a,SARS-CoV-2_Wuhan-Hu-1,SARS-CoV-2_Wuhan-Hu-1,13100,no gaps,13100,100.000000
1,ORF1a,SARS-CoV-2_Wuhan-Hu-1,RaTG13,12579,no gaps,13097,96.044896
2,ORF1a,SARS-CoV-2_Wuhan-Hu-1,RaTG13,12579,no gaps,13097,96.044896
3,ORF1a,SARS-CoV-2_Wuhan-Hu-1,SARS-CoV-1,9881,no gaps,12998,76.019388
4,ORF1a,SARS-CoV-2_Wuhan-Hu-1,SARS-CoV-1,9881,no gaps,12998,76.019388
...,...,...,...,...,...,...,...
39361,nsp16,RShSTT182,GD_Pangolin,820,with gaps,894,91.722595
39362,nsp16,RShSTT200,RShSTT200,894,with gaps,894,100.000000
39363,nsp16,RShSTT200,GD_Pangolin,820,with gaps,894,91.722595
39364,nsp16,RShSTT200,GD_Pangolin,820,with gaps,894,91.722595
